We want to learn $\lambda$ values separated by an order of magnitude and show that the errors scale correctly for both. 

In [1]:
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, Optim, GalacticOptim
using Printf

# ---------------------
# Model definition
# ---------------------
struct Phi4Model1D
    lattice_size::Int   # Number of sites (N)
    lambda::Float64     # Quartic coupling
    m2::Float64         # Mass term squared
end

# Energy function for 1D phi^4
function calculate_energy_1d(phi::Vector{T}, model::Phi4Model1D) where T<:Real
    N = length(phi)
    kinetic = 0.0
    potential = 0.0
    for i in 1:N
        # Periodic neighbor to the right
        i_right = (i % N) + 1
        kinetic += 0.5 * (phi[i] - phi[i_right])^2
        potential += 0.5 * model.m2 * phi[i]^2 + 0.25 * model.lambda * phi[i]^4
    end
    return kinetic + potential
end

# Target density for HMC (LogDensityProblems interface)
struct Phi4TargetDensity1D
    dim::Int
    model::Phi4Model1D
end

function LogDensityProblems.logdensity(p::Phi4TargetDensity1D, phi::Vector{T}) where T<:Real
    return -calculate_energy_1d(phi, p.model)
end
LogDensityProblems.dimension(p::Phi4TargetDensity1D) = p.model.lattice_size
LogDensityProblems.capabilities(::Type{Phi4TargetDensity1D}) = LogDensityProblems.LogDensityOrder{1}()

# HMC sampling
function generate_phi4_data_hmc_1d(model::Phi4Model1D, n_samples::Int, n_adapts::Int=2000)
    N = model.lattice_size
    target = Phi4TargetDensity1D(N, model)
    initial_phi = randn(N)
    metric = DiagEuclideanMetric(N)
    hamiltonian = Hamiltonian(metric, target, ForwardDiff)
    ε0 = find_good_stepsize(hamiltonian, initial_phi)
    integrator = Leapfrog(ε0)
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))
    samples_vec, stats = sample(hamiltonian, kernel, initial_phi, n_samples, adaptor, n_adapts; progress=false)
    return [copy(s) for s in samples_vec]
end

# Score matching objective (no a)
function score_matching_objective_1d(lambda::Float64, m2::Float64, data::Vector{Vector{Float64}})
    N = length(data[1])
    n_samples = length(data)
    obj = 0.0
    for phi in data
        for i in 1:N
            x = phi[i]
            # Periodic neighbors
            xl = phi[mod1(i-1, N)]
            xr = phi[mod1(i+1, N)]
            # Kinetic Laplacian
            kinetic = -(2*x - xl - xr)
            score = kinetic - (m2*x + lambda*x^3)
            sprime = -(m2 + 3*lambda*x^2 + 2)
            obj += 0.5*score^2 + sprime
        end
    end
    return obj / n_samples
end

# Estimate λ and m2 by score matching
function estimate_parameters_score_matching_1d(data::Vector{Vector{Float64}})
    f(x) = score_matching_objective_1d(x[1], x[2], data)
    initial = [0.1, 0.1]
    ps = ParticleSwarm()
    res = optimize(f, initial, ps)
    λ̂, m2̂ = res.minimizer
    @printf("Final objective = %.5e; estimated λ=%.5f, m²=%.5f\n", res.minimum, λ̂, m2̂)
    return λ̂, m2̂
end


"""

# Example workflow
lattice_size = 8
λ_true = 0.2
m2_true = 0.0
model = Phi4Model1D(lattice_size, λ_true, m2_true)
n_samples = 10000
@printf("Sampling %d with HMC…\n", n_samples)
samps = generate_phi4_data_hmc_1d(model, n_samples)
λ_est, m2_est = estimate_parameters_score_matching_1d(samps)
@printf("True λ=%.3f, est=%.3f; true m²=%.3f, est=%.3f\n", λ_true, λ_est, m2_true, m2_est)
"""


"\n# Example workflow\nlattice_size = 8\nλ_true = 0.2\nm2_true = 0.0\nmodel = Phi4Model1D(lattice_size, λ_true, m2_true)\nn_samples = 10000\n@printf(\"Sampling %d with HMC…\n\", n_samples)\nsamps = generate_phi4_data_hmc_1d(model, n_samples)\nλ_est, m2_est = estimate_parameters_score_matching_1d(samps)\n@printf(\"True λ=%.3f, est=%.3f; true m²=%.3f, est=%.3f\n\", λ_true, λ_est, m2_true, m2_est)\n"

Define the two models

In [ ]:
lattice_size = 10
model1 = Phi4Model1D(lattice_size, 0.2, 0.0)
model2 = Phi4Model1D(lattice_size, 2.0, 0.0)

Phi4Model1D(10, 2.0, 0.0)

Code to generate samples for each sample size for various runs and learn $\lambda$ and store errors 

In [ ]:
using Statistics, Printf
using Base.Threads

sizes = [10_000 * 2^i for i in 0:floor(Int, log2(2_000_000/20_000))]
nrep = 50

# Store *all* λ estimates for error bar calculation
lambdas1 = Vector{Vector{Float64}}(undef, length(sizes))
lambdas2 = Vector{Vector{Float64}}(undef, length(sizes))

for (k, N) in enumerate(sizes)
    # Preallocate arrays for all reps
    these_lambdas1 = Vector{Float64}(undef, nrep)
    these_lambdas2 = Vector{Float64}(undef, nrep)
    
    @printf("→ Running N = %7d  ", N)
    
    Threads.@threads for rep in 1:nrep
        data1 = generate_phi4_data_hmc_1d(model1, N)
        data2 = generate_phi4_data_hmc_1d(model2, N)
        l1, m1 = estimate_parameters_score_matching_1d(data1)
        l2, m2 = estimate_parameters_score_matching_1d(data2)
        these_lambdas1[rep] = l1
        these_lambdas2[rep] = l2
    end
    
    lambdas1[k] = these_lambdas1
    lambdas2[k] = these_lambdas2
    m_err1 = mean(abs.(these_lambdas1 .- 0.2))
    m_err2 = mean(abs.(these_lambdas2 .- 2.0))
    @printf("mean |λ̂–λ₀| = %.3e  (small, λ₀=0.2)\n", m_err1)
    @printf("mean |λ̂–λ₀| = %.3e  (large, λ₀=2.0)\n", m_err2)
end

# Now you can compute mean and stddev error bars for each size:
mean_abs_errors1 = [mean(abs.(ls .- 0.2)) for ls in lambdas1]
std_abs_errors1  = [std(abs.(ls .- 0.2)) / sqrt(length(ls)) for ls in lambdas1]
mean_abs_errors2 = [mean(abs.(ls .- 2.0)) for ls in lambdas2]
std_abs_errors2  = [std(abs.(ls .- 2.0)) / sqrt(length(ls)) for ls in lambdas2]

# (mean_abs_errors1, std_abs_errors1) now give you means and error bars for plotting.
